In [1]:
import h3
import geopandas as gpd
import rasterio
from rasterio.enums import Resampling
import rasterstats as rs
from shapely.geometry import Polygon

In [2]:
def reclassify_raster(input_raster, output_raster):
    # Otwieranie pliku rasterowego
    with rasterio.open(input_raster) as src:
        data = src.read(1)  # Wczytanie pierwszej warstwy

        # Reklasyfikacja danych
        data[data == 2] = 0
        data[data == 1] = 1  # Zamiana wartości 2 na 1

        # Zapis nowego pliku z zaktualizowanymi danymi
        with rasterio.open(
            output_raster,
            'w',
            driver='GTiff',
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            dst.write(data, 1)


def zonal_stats(raster_path, hex_gdf, field):   

    with rasterio.open(raster_path) as raster_interpolate:
        # Masked = True sets no data values to np.nan if they are in the metadata
        raster_data = raster_interpolate.read(1, masked=True)
        raster_meta = raster_interpolate.profile

    hexagon_zonal = rs.zonal_stats(hex_gdf,
                                    raster_data,
                                    nodata=-999,
                                    affine=raster_meta['transform'],
                                    geojson_out=True,
                                    copy_properties=True,
                                    stats="count min mean max median")

    hexagons_zonal = gpd.GeoDataFrame.from_features(hexagon_zonal)
    hexagons_zonal[field] = hexagons_zonal['mean']
    # hexagons_zonal = hexagons_zonal[['geometry', field]]
    return hexagons_zonal



def load_gdf_and_generate_h3(file_path, resolution=9):
    # Wczytanie GeoDataFrame
    gdf = gpd.read_file(file_path)

    # Lista na przechowywanie heksagonów H3
    hexagons = []

    # Przetwarzanie każdego poligonu w GDF
    for index, row in gdf.iterrows():
        # Uzyskanie geometrii każdego poligonu
        poly = row['geometry']

        # Generowanie heksagonów H3 dla danego poligonu
        hex_ids = h3.polyfill(poly.__geo_interface__, resolution, geo_json_conformant=True)

        # Tworzenie poligonów z heksagonów
        hex_polygons = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in hex_ids]

        # Dodawanie do listy
        hexagons.extend(hex_polygons)

    # Tworzenie nowego GeoDataFrame
    hex_gdf = gpd.GeoDataFrame(geometry=hexagons)

    return hex_gdf

In [3]:
# Przykład użycia
file_path = r'd:\GIS\GIS_projects\2024_jakarta\johor\johor_admin.gpkg'
hex_gdf = load_gdf_and_generate_h3(file_path)
print(hex_gdf)

                                               geometry
0     POLYGON ((103.93731 1.64655, 103.93686 1.64841...
1     POLYGON ((103.81427 1.52543, 103.81382 1.52730...
2     POLYGON ((103.62975 1.37947, 103.62930 1.38133...
3     POLYGON ((103.74355 1.51011, 103.74309 1.51197...
4     POLYGON ((103.70896 1.61727, 103.70851 1.61914...
...                                                 ...
8664  POLYGON ((103.91794 1.54036, 103.91749 1.54223...
8665  POLYGON ((103.87837 1.58963, 103.87792 1.59150...
8666  POLYGON ((103.74833 1.59768, 103.74788 1.59954...
8667  POLYGON ((103.99013 1.57094, 103.98968 1.57280...
8668  POLYGON ((103.90396 1.53377, 103.90351 1.53563...

[8669 rows x 1 columns]


In [13]:
# Przykład użycia funkcji reklas
for year in [2016, 2018, 2020, 2022, 2024]:
    input_raster_path = fr'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\{year}_21042024_11classes.tif'
    output_raster_path = fr'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\{year}_21042024_11classes_rekl.tif'
    reclassify_raster(input_raster_path, output_raster_path)
    hex_gdf = zonal_stats(output_raster_path, hex_gdf, year)

In [7]:
# # TODO time series analysis
# przepusci wzrost tylko wtedy jesli mamy wiecej niz 10 pp
# albo wiecje niz 50% zmiany
# albo wiecej niz 50% ale nie mniej niz 10 pp

In [15]:
hex_gdf

,geometry,min,max,mean,count,median,2016,2018,2020,2022,2024
0,"POLYGON ((103.93731 1.64655, 103.93686 1.64841...",0.0,0.0,0.000000,1204,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,"POLYGON ((103.81427 1.52543, 103.81382 1.52730...",0.0,1.0,0.359401,1202,0.0,0.533278,0.351082,0.475874,0.454243,0.359401
2,"POLYGON ((103.62975 1.37947, 103.62930 1.38133...",0.0,0.0,0.000000,1199,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,"POLYGON ((103.74355 1.51011, 103.74309 1.51197...",0.0,1.0,0.621464,1202,1.0,0.707987,0.659734,0.769551,0.715474,0.621464
4,"POLYGON ((103.70896 1.61727, 103.70851 1.61914...",0.0,1.0,0.118672,1205,0.0,0.072199,0.206639,0.224066,0.178423,0.118672
...,...,...,...,...,...,...,...,...,...,...,...
8664,"POLYGON ((103.91794 1.54036, 103.91749 1.54223...",0.0,0.0,0.000000,1199,0.0,0.000000,0.000000,0.353628,0.000000,0.000000
8665,"POLYGON ((103.87837 1.58963, 103.87792 1.59150...",0.0,1.0,0.010833,1200,0.0,0.030000,0.005833,0.013333,0.050833,0.010833
8666,"POLYGON ((103.74833 1.59768, 103.74788 1.59954...",0.0,0.0,0.000000,1198,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8667,"POLYGON ((103.99013 1.57094, 103.98968 1.57280...",0.0,0.0,0.000000,1200,0.0,0.005000,0.000000,0.000000,0.000000,0.000000


In [21]:
# Funkcja, która sprawdzi warunki
def calculate_new_column(year, prev_year):
    if year >= prev_year * 1.2 and year >= prev_year + 0.1:
        return year
    else:
        return prev_year

# TODOS
# greenfield - dodatkowa kategoria- od zera jak jest cokolwiek pewnego

In [17]:
hex_gdf
# TODO - for initial point - make a median of 2016 and 2018?
## - if 2016 bigger than 2018 - then apply mean of 2016 and 2018, then process

,geometry,min,max,mean,count,median,2016,2018,2020,2022,2024
0,"POLYGON ((103.93731 1.64655, 103.93686 1.64841...",0.0,0.0,0.000000,1204,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,"POLYGON ((103.81427 1.52543, 103.81382 1.52730...",0.0,1.0,0.359401,1202,0.0,0.533278,0.351082,0.475874,0.454243,0.359401
2,"POLYGON ((103.62975 1.37947, 103.62930 1.38133...",0.0,0.0,0.000000,1199,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,"POLYGON ((103.74355 1.51011, 103.74309 1.51197...",0.0,1.0,0.621464,1202,1.0,0.707987,0.659734,0.769551,0.715474,0.621464
4,"POLYGON ((103.70896 1.61727, 103.70851 1.61914...",0.0,1.0,0.118672,1205,0.0,0.072199,0.206639,0.224066,0.178423,0.118672
...,...,...,...,...,...,...,...,...,...,...,...
8664,"POLYGON ((103.91794 1.54036, 103.91749 1.54223...",0.0,0.0,0.000000,1199,0.0,0.000000,0.000000,0.353628,0.000000,0.000000
8665,"POLYGON ((103.87837 1.58963, 103.87792 1.59150...",0.0,1.0,0.010833,1200,0.0,0.030000,0.005833,0.013333,0.050833,0.010833
8666,"POLYGON ((103.74833 1.59768, 103.74788 1.59954...",0.0,0.0,0.000000,1198,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8667,"POLYGON ((103.99013 1.57094, 103.98968 1.57280...",0.0,0.0,0.000000,1200,0.0,0.005000,0.000000,0.000000,0.000000,0.000000


In [26]:
hex_gdf['2016_corr'] = hex_gdf[2016]

for year, prev_year in zip([2018, 2020, 2022, 2024], [2016,2018,2020,2022]):
    print(year, prev_year)
    # hex_gdf['2024_v2'] = hex_gdf.apply(calculate_new_column, axis=1)
    hex_gdf[f'{year}_corr'] = hex_gdf.apply(lambda row: calculate_new_column(row[year], row[f'{prev_year}_corr']), axis=1)
    

2018 2016
2020 2018
2022 2020
2024 2022


In [31]:
hex_gdf
hex_gdf[['geometry', 
         '2016_corr', '2018_corr', '2020_corr', '2022_corr', '2024_corr']]. \
    to_file(r'd:\GIS\GIS_projects\2024_jakarta\johor\h3\h3_all.shp')

In [12]:
# TODO
# wymyslec jakas sensowna metryke na timeseries
#    # - np pow heksa *1/10 lub * 1/4 lub * 1/2 lub * 1 - w zaleznosci ile % zabudowy w heksie
# inne podejscie: zabudowany lub nie : > 15% ?

# co mozna robic? wyczyscic recznie rastry - klasyfikacje nr 1 i nr ostatnia
# daty posrednie - postarac sie uchwycic zmiane tempa zabudowy
# zliczanie powierzchni excavation YoY

# dodac overture buildings? - 